In [1]:
import _nb_utils
%load_ext autoreload
%autoreload 2

In [2]:
from config import CONFIG
from prompter import *
from data import *
from utils import *
from model import GenParams, GoogleAIStudioMLLM, OllamaMLLM
from viz import display_prompt

In [3]:
model_name = "gemma3:12b-it-qat"

vlm = OllamaMLLM(model_name)

In [4]:
by_model = "LRASPP_MobileNet_V3"

In [5]:
gen_params = GenParams(
    seed=CONFIG["seed"],
    temperature=0.1
)

In [6]:
prompt_blueprint={
        "context": "default",
        "color_map": "default",
        "input_format": "sep_ovr_original",
        "task": "default",
        "output_format": "default",
        "support_set_intro": "default",
        "support_set_item": "default",
        "query": "default",
}

In [7]:
fast_prompt_builder = FastPromptBuilder(
    seed=CONFIG["seed"],
    prompt_blueprint=prompt_blueprint,
    by_model=by_model,
    alpha=0.6,
    class_map=CLASS_MAP,
    color_map=COLOR_MAP_DICT,
    image_size=224,
    sup_set_img_idxs=[16],
    str_formats=None,
    prs_mask_paths=get_mask_prs_path(by_model=by_model),
    jsonl_save_path=None
)

In [8]:
epoch_idxs = [x for x in list(range(80)) if x not in fast_prompt_builder.sup_set_img_idxs]
len(epoch_idxs)

79

In [9]:
cs_inference_prompts = fast_prompt_builder.build_cs_inference_prompts(epoch_idxs)
len(cs_inference_prompts)

79

In [14]:
# display_prompt(cs_inference_prompts[0][15])

In [15]:
exp_name = "after_fast_prompter"

answer_path = get_answer_prs_root_path("LRASPP_MobileNet_V3", "class-splitted") / model_name / exp_name / "SepMasks_Ovr_1fs_2.jsonl"
eval_path = get_eval_prs_root_path("LRASPP_MobileNet_V3", "class-splitted") / model_name / exp_name / "SepMasks_Ovr_1fs_2.jsonl"
print(answer_path)
print(eval_path)

/home/olivieri/exp/my_data/by_model/LRASPP_MobileNet_V3/class-splitted/answer_prs/gemma3:12b-it-qat/after_fast_prompter/SepMasks_Ovr_1fs_2.jsonl
/home/olivieri/exp/my_data/by_model/LRASPP_MobileNet_V3/class-splitted/eval_prs/gemma3:12b-it-qat/after_fast_prompter/SepMasks_Ovr_1fs_2.jsonl


In [16]:
append_many_to_jsonl(answer_path, [{"state": fast_prompt_builder.get_state()} | {"vlm": f"{vlm.__class__.__name__}:{vlm.model}"}]) 

In [17]:
epoch_answer_list = []

epoch_answer_list += await vlm.predict_many_class_splitted(
    cs_inference_prompts,
    epoch_idxs,
    gen_params=gen_params,
    jsonl_save_path=answer_path,
    only_text=True,
    splits_in_parallel=False,
    batch_size=None,
)

100%|██████████| 79/79 [26:09<00:00, 19.86s/item]
